<br>

<h1>
    <center>
        Estimating $\pi$ using Various Methods
    </center>
</h1>

## Table of Contents


0. [Define Problem](#0.-Define-Problem)
1. [Import Libraries](#1.-Import-Libraries)
2. [Estimate $\pi$ using Taylor Series Expansion](#2.-Estimate-$\pi$-using-Taylor-Series-Expansion) 
    * 2.1 Define Functions
        * GenerateTaylorSeries
    * 2.2 Perform Computations
    * 2.3 Construct Records DataFrame
3. [Estimate $\pi$ using Monte Carlo Simulation with Numpy (NumpyMC)](#3.-Estimate-$\pi$-using-Monte-Carlo-Simulation-with-Numpy)
    * 3.1 Perform Numpy Computations
    * 3.2 Construct Records DataFrame
4. [Estimate $\pi$ using Quantum Monte Carlo Simulation (QCMC)](#4.-Estimate-$\pi$-using-Quantum-Monte-Carlo-Simulation) 
    * 4.1 Define Functions
        * CreateCircuit
        * GenerateRandomNumbers_QC
        * NormalizeNumbers
    * 4.2 Perform Computations
    * 4.3 Construct Performance DataFrame
5. [Analysis of Performance](#5.-Analysis-of-Performance)
    * 5.1 Construct $\pi$ Approximation Performance DataFrame
    * 5.2 Save Performance DataFrame as csv
    * 5.3 Plot Sample Size vs. $\pi$ Approximation
    * 5.4 Plot Sample Size vs. $\pi$ Approximation Error
    * 5.5 Plot Computational Time vs. $\pi$ Approximation Error
    * 5.6 Plot  $\pi$ Approximation Error vs. $Log_{10}$(Population)
    * 5.7 Measure Corrilation between $\pi$ Approximation Error and Sample Size
6. [Conclusion - Which is better?](#6.-Conclusion)

### 0. Define Problem

<br>

The below code is dedicated to approximating $\pi$ using various computational methods. The famously irrational number can be approximated using iterative methods envolving the __Taylor Series__ approximation and __Monte Carlo__ simulation methods. Each of the respective methods is reliant on the use of iterative computations to approximate $\pi$ and with each sucessive iteration get closer to $\pi$. the follwing Cod will test various population sizes against different algorithms to test for both speed and acurracy.


To test the fidelity of each algorithm populations of various sizes will be tested against different population sizes ranging in value of $10^{2}, 10^{3}, 10^{4}, 10^{5}, 10^{6}, 10^{7}, 10^{8}$. With larger popoulations, more iterations can be conducted and it is hypothesized that population size and error measured between $\pi$ and the associated approximation measured as:

$$error = | \pi - approximation|$$

are negatively corrilated such that, more iterations cause less error.


However dependening on the method used, the difference between the true value of $\pi$ and its approximated value should become smaller, faster with successive iterations. The below code will attempt to secondarily measure which method is better via metrics associated accuracy and speed.


<br>

#### Approximating $\pi$ vai the Taylor Series Expantion:

$$\pi = 4 \sum_{i=1}^{n} \frac{(-1)^{n}}{2n-1}$$

$$ \pi = 4 \left( 1 - \frac{1}{3} + \frac{1}{5} - \frac{1}{7} + \frac{1}{9} ... \frac{1}{n} \right) = 3.1415926...$$

<br>

#### Aproximate $\pi$ via Monte Carlo Simulation:


$$ \pi \sim \frac{ || \sqrt{(x^{2} + y^{2})} || < 1 }{n}$$

<br>

![MC_Simulation](Monte_Carlo_Approximation.gif)

### 1. Import Libraries

In [ ]:
# import visulaization tool
import seaborn as sns
import matplotlib.pyplot as plt

# import numerical processing tool
import numpy as np
np.random.seed(1234)

# import quantum simulation tools
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import BasicAer, execute
import pylatexenc

# data handeling
import pandas as pd

# measure time/duration 
import time

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

### 2. Estimate $\pi$ using Taylor Series Expansion

$$\pi = 4 \sum_{i=1}^{n} \frac{(-1)^{n}}{2n-1} = 4  \left( 1 - \frac{1}{3} + \frac{1}{5} - \frac{1}{7} + \frac{1}{9} ... \frac{1}{n }\right)$$


#### 2.1 Define Functions - GenerateTaylorSeries

In [ ]:
def GenerateTaylorSeries(x):
    """
    Input:
        x | int | numebr of iterations to approximate taylor series expansion pi/4
    Output:
        pi_aproximation | float | approximation of pi        
    """
        
    iteration_value = [(1**(2*i+1)*(-1)**i)/(2*i+1) for i in np.arange(x)]
    summation = np.sum(iteration_value)
    pi_approximation = 4 * summation
        
    return pi_approximation

#### 2.2 Perform Computations


In [ ]:
# establish lists to hold recorded values
pi_approximation_list = []
population_list = []
duration = [] 

for sample_size in [int(1e2), int(1e3), int(1e4), int(1e5), int(1e6), int(1e7), int(1e8)]:

    print(sample_size)
        
    # measure start time
    start = time.time()   
    
    # generate taylor series
    pi_approximation = GenerateTaylorSeries(x = sample_size)
    
    # measure stop time
    stop = time.time()
    
    # record keeping
    duration.append(stop-start)        
    pi_approximation_list.append(pi_approximation)
    population_list.append(sample_size)
    

#### 2.3 Construct Records DataFrame

In [ ]:
TaylorSeries_Performance = (
    pd.DataFrame(
        data = list(zip(pi_approximation_list, population_list, duration)),
        columns = ['pi_approximatation', 'population', 'duration']
    )
    .assign(        
        error = lambda x: x['pi_approximatation'] - np.pi
    )   
)

display(TaylorSeries_Performance)

### 3. Estimate $\pi$ using Monte Carlo Simulation with Numpy

#### 3.1 Perform Numpy Computations

In [ ]:
# establish lists to hold recorded values
pi_approximation_list = []
population_list = []
duration = [] 

for sample_size in [int(1e2), int(1e3), int(1e4), int(1e5), int(1e6), int(1e7), int(1e8)]:

    print(sample_size)
        
    # measure start time
    start = time.time()   
    
    # generate x values
    x = np.random.random_sample(sample_size)
    # generate y values
    y = np.random.random_sample(sample_size)
    
    # create MC 
    mc_values = np.sqrt(np.square(x) + np.square(y))
    pi_approximation = 4*len(mc_values[mc_values <= 1]) / len(x)
    
    # measure stop time
    stop = time.time()
    
    # record keeping
    duration.append(stop-start)        
    pi_approximation_list.append(pi_approximation)
    population_list.append(sample_size)

#### 3.2 Construct Records DataFrame

In [ ]:
Numpy_Performance = (
    pd.DataFrame(
        data = list(zip(pi_approximation_list, population_list, duration)),
        columns = ['pi_approximatation', 'population', 'duration']
    )
    .assign(        
        error = lambda x: x['pi_approximatation'] - np.pi
    )   
)

display(Numpy_Performance)

### 4. Estimate $\pi$ using Quantum Monte Carlo Simulation

#### 4.1 Define Functions - CreateCircuit

In [ ]:
def CreateCircuit(bits):
    """
    Input:
        bits | int | defines how many q-bits the circuit involves
    Output:
        circuit | qiskit.circuit | establish a circuit to randomly generate 8-bit binary numbers
        circuit_diagram | matplotlib.figure | figure that depicts the q-bit to c-bit figure
    """    
    
    # define quantum register | classical register
    q = QuantumRegister(bits);  c = ClassicalRegister(bits)
    
    # define circuit
    circuit = QuantumCircuit(q, c)
    
    # use for loops to add q-bits and hadamar gates to the existing circuit
    for j in range(bits): circuit.h(q[j])
    
    # add measurements to transfer from q-bit to classical-bit
    circuit.measure(q, c)
    
    # draw circuit
    circuit_drawing = circuit.draw(output='mpl')
    
    # return circuit and drawing
    #return (circuit, circuit_drawing)
    return circuit

#### 4.1 Define Functions - GenerateRandomNumbers_QC

In [ ]:
def GenerateRandomNumbers_QC(circuit, sample_size):
    """
    Input:
        circuit | qiskit.circuit | establish a circuit to randomly generate 8-bit binary numbers
        sample_size | int | number of iterations to generate 8-bit numbers
    Output:
        numbers | list(int) | list of binary numbers generated by the circuit
    """   
    
    # generate binary numbers
    binary_numbers = execute(
        experiments = circuit,
        backend = BasicAer.get_backend('qasm_simulator'),
        shots = sample_size,
        memory = True
    )
    
    # convert binary numbers to integers
    numbers = np.array([int(i, 2) for i in binary_numbers.result().get_memory()])
    
    return numbers

#### 4.1 Define Functions - NormalizeNumbers

In [ ]:
def NormalizeNumbers(numbers):
    """
    Input:
        numbers | list(int) | list of binary numbers generated by the circuit
    Output:
        normalized_values | list(float) | list of normalized numbers between [0,1]
    """
    minimum = min(numbers)
    maximum = max(numbers)
    
    normalized_values = np.array([(num - maximum) / (maximum - minimum) for num in numbers])
    
    return normalized_values

#### 4.2 Perform Computations

In [ ]:
# establish circute
bits = 8
circuit = CreateCircuit(bits = bits)

# establish lists to hold recorded values
pi_approximation_list = []
population_list = []
duration = [] 

# perform iterations    
for sample_size in [int(1e2), int(1e3), int(1e4), int(1e5), int(1e6), int(1e7), int(1e8)]:

    print(sample_size)
        
    # measure start time
    start = time.time()   
    
    # generate x values
    x = NormalizeNumbers(GenerateRandomNumbers_QC(circuit = circuit, sample_size = sample_size))
    # generate y values
    y = NormalizeNumbers(GenerateRandomNumbers_QC(circuit = circuit, sample_size = sample_size))
    
    # create MC
    mc_values = np.sqrt(np.square(x) + np.square(y))
    pi_approximation = 4*len(mc_values[mc_values <= 1]) / len(x)
    
    # measure stop time
    stop = time.time()
    
    # record keeping
    duration.append(stop-start)        
    pi_approximation_list.append(pi_approximation)
    population_list.append(sample_size)

#### 4.3 Construct Records DataFrame

In [ ]:
QuantumComputing_Performance = (
    pd.DataFrame(
        data = list(zip(pi_approximation_list, population_list, duration)),
        columns = ['pi_approximatation', 'population', 'duration']
    )
    .assign(        
        error = lambda x: x['pi_approximatation'] - np.pi
    )   
)

display(QuantumComputing_Performance)

### 5. Analysis of Performance

#### 5.1 Construct $\pi$ Approximation Performance DataFrame

In [ ]:
TaylorSeries_Performance     = TaylorSeries_Performance.assign(method = 'TaylorSeries')
Numpy_Performance            = Numpy_Performance.assign(method = 'NumpyMC')
QuantumComputing_Performance = QuantumComputing_Performance.assign(method = 'QCMC')

Performance_DataFrame = (
    pd.concat(
        [TaylorSeries_Performance, Numpy_Performance, QuantumComputing_Performance]
    )
    [['method', 'population', 'duration', 'pi_approximatation', 'error']]
)

####  5.2 Save Performance DataFrame as csv

In [ ]:
Performance_DataFrame.to_csv("./Pi_Approximation_Performance_Record.csv", index = False)

#### 5.3 Plot Sample Size vs. $\pi$ Approximation

In [ ]:
fig, ax0 = plt.subplots()

for algorithm in ['TaylorSeries', 'NumpyMC', 'QCMC']:
    (
        Performance_DataFrame
        .query(f"method == '{algorithm}'")
        .assign(log_pop = lambda x: np.log10(x['population']))
        .plot(
            x = 'log_pop',
            y = 'pi_approximatation',
            ax = ax0,
            alpha = 0.6,
            style = "o--",            
            markersize = 10,            
            xlabel = "log10 Computational Iterations",
            ylabel = "Approximation to Pi",
            title = "Iterative Approximation of Pi",
            figsize = (16, 5),
        )
    )
    
ax0.legend(['TaylorSeries', 'NumpyMC', 'QCMC'])
plt.axhline(
    y = np.pi,
    xmin = 0, xmax = Performance_DataFrame.population.max(),
    color = 'red',
    linewidth = 2,
    linestyle = 'dotted'
)

#### 5.4 Plot $log_{10}$(Durations) vs. $\pi$ Approximation Error

In [ ]:
fig, ax0 = plt.subplots()

for algorithm in ['TaylorSeries', 'NumpyMC', 'QCMC']:
    (
        Performance_DataFrame
        .query(f"method == '{algorithm}'")
        .assign(log_pop = lambda x: np.log10(x['population']))
        .plot(
            x = 'log_pop',
            y = 'error',
            ax = ax0,
            alpha = 0.6,
            style = "o--",
            markersize = 10,            
            xlabel = "log10 Computational Iterations",
            ylabel = "Error",
            title = "Error Approximation of Pi vs Number of Computational Samples",
            figsize = (16, 5),
        )
    )
    
ax0.legend(['TaylorSeries', 'NumpyMC', 'QCMC'])
plt.axhline(
    y = 0,
    xmin = 0, xmax = Performance_DataFrame.population.max(),
    color = 'red',
    linewidth = 2,
    linestyle = 'dotted'
)

#### 5.5 Plot $log_{10}$(Durations) vs. $\pi$ Approximation Error

In [ ]:
fig, ax0 = plt.subplots()

for algorithm in ['TaylorSeries', 'NumpyMC', 'QCMC']:
    (
        Performance_DataFrame
        .query(f"method == '{algorithm}'")
        .assign(
            log_pop = lambda x: np.log10(x['population']),
            log_duration = lambda x: np.log(x['duration'])
        )
        .sort_values('log_duration')
        .plot(
            x = 'log_duration',
            y = 'error',
            ax = ax0,
            alpha = 0.6,
            style = "o--",            
            markersize = 10,
            xlabel = "log10 Computational Duration log10(seconds)",
            ylabel = "Error Approximation to Pi",
            title = "Copmutational Duration and Pi Approximation Error",
            figsize = (16, 5),
        )
    )
    
ax0.legend(['TaylorSeries', 'NumpyMC', 'QCMC'])
plt.axhline(
    y = 0,
    xmin = 0, xmax = Performance_DataFrame.population.max(),
    color = 'red',
    linewidth = 2,
    linestyle = 'dotted'
)

#### 5.6 Plot  $\pi$ Approximation Error vs. $Log_{10}$(Population)

In [ ]:
fig, ax0 = plt.subplots()

for algorithm in ['TaylorSeries', 'NumpyMC', 'QCMC']:
    (
        Performance_DataFrame
        .query(f"method == '{algorithm}'")
        .assign(
            log_pop = lambda x: np.log10(x['population']),
            log_duration = lambda x: np.log(x['duration']),
            abs_error = lambda x: abs(x['error'])
        )
        .sort_values('log_pop')
        .plot(
            x = 'log_pop',
            y = 'abs_error',
            ax = ax0,
            alpha = 0.6,
            style = "o--",            
            markersize = 10,
            xlabel = "log10 Computational Duration log10(seconds)",
            ylabel = "Error Approximation to Pi",
            title = "Duration anad Pi Approximation Error",
            figsize = (16, 5),
        )
    )
    
ax0.legend(['TaylorSeries', 'NumpyMC', 'QCMC'])
plt.axhline(
    y = 0,
    xmin = 0, xmax = Performance_DataFrame.population.max(),
    color = 'red',
    linewidth = 2,
    linestyle = 'dotted'
)

#### 5.7 Measure Corrilation between Pi Approximation Error and Sample Size

In [ ]:
for algorithm in ['TaylorSeries', 'NumpyMC', 'QCMC']:
    display(
        f"Algorithm: {algorithm}",
        (
            Performance_DataFrame
            .query(
                f"method == '{algorithm}'"
            )
            .assign(
                log_pop = lambda x: np.log10(x['population']),
                log_duration = lambda x: np.log(x['duration']),
                abs_error = lambda x: abs(x['error'])
            )        
            [['population', 'abs_error']]
            .corr()
        )
    )

### 6. Conclusion

<br>

#### Bottom Line Up Front (BLUF):

Ranking the various algorithms by speed and accuracy:

1. TaylorSeries
2. NumpyMC
3. QCMC

<br>

#### Analysis:

The series method of approxiamting $\pi$ only requires one algorithmicaly generated list with which to perform computations. In constrast each of the _Monte Carlo Approximation_ methods requires two of the aforementioned algorithmicaly generated lists and should therefore be expected, and proven, to be less effecient that the aformentioned _Taylor Series Expansion_ method. Between the two various of the Monte Carlo Approximation method, the Numpy utilzation method perofrmed faster and more accurately for population sizes greater than $10^{2}$ as compared to the Quantum Computing method. Incidently the corrilation between population size and absolute value of the Approximation Error are indeed negtively corrilated with various degrees of strength listed below:

<br>

| Method       | Corrilation |
|--------------|-------------|
| TaylorSeries | -0.211531   |
| NumpyMC      | -0.324324   |
| QCMC         | -0.195622   | 

<br>

Although weakly corrilated the NumpyMC algorithm is almost twices a negatively corrilated as the QCMC algorithm futher providign evidence to the notation that the NumpyMC algorithm is faster and more accruate the the QCMC algorithm.

In [ ]:
def ConditionallyColor(x):
    
    """
    Input:
        x | list(float) | monte carlo simulation values
    Output:
        color | str | color associated with monte carlo visualization
    """
    
    
    if abs(x) <= 1:
        color = 'blue'
    else:
        color = 'red'
        
    return color

In [ ]:
# establish lists to hold recorded values
pi_approximation_list = []
population_list = []
duration = [] 

for sample_size in [int(1e2), int(1e3), int(1e4), int(1e5), int(1e6), int(1e7), int(1e8)]:
    
    # generate x values
    x = np.random.random_sample(sample_size)
    # generate y values
    y = np.random.random_sample(sample_size)
    
    # create MC 
    mc_values = np.sqrt(np.square(x) + np.square(y))
    pi_approximation = 4*len(mc_values[mc_values <= 1]) / len(x)
    
    
    temp = (
        pd.DataFrame(
            data = list(zip(x, y)),
            columns = ['x', 'y']
        )
        .assign(
            mc_values = lambda x: np.sqrt(np.square(x['x']) + np.square(x['y'])),
            color = lambda x: x['mc_values'].map(ConditionallyColor)        
        )
    )
    
    (
        temp
        .plot(
            x = 'x',
            y = 'y',
            alpha = 0.2,
            kind = 'scatter',
            color = temp['color'],
            title = f"n = {sample_size} | pi = {pi_approximation}",
            figsize = (8, 8)
        )
        .get_figure()
        .savefig(f"NumpyMC Visualization n_{sample_size}.png")
    )